In [1]:
import sys
sys.path.append("../")

In [10]:
from utils import train_nli
from data import load_gyafc
from torch.utils.data import Dataset, DataLoader
import torch
from transformers import (DebertaTokenizer,
                          DebertaForSequenceClassification,
                          AutoModelForSequenceClassification,  
                          AutoTokenizer,
                          Trainer,TrainingArguments)
import os
import json
from tqdm import tqdm

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")

In [4]:
def get_test_dataset(model_name):
    tr_val_test_datasets = load_gyafc(model_name, toy = False)
    _, _, test_dataset = tr_val_test_datasets
    return test_dataset

In [5]:
def get_model_type(model_name):
    if "deberta-large" in model_name:
        return "microsoft/deberta-large"
    elif "deberta-base" in model_name:
        return "microsoft/deberta-base"

In [6]:
test_dataset_dict = {}
for mt in ["microsoft/deberta-large","microsoft/deberta-base"]:
    test_dataset_dict[mt] = get_test_dataset(mt)

In [7]:
# os.listdir(trained_models_fld)

In [12]:
trained_models_fld = "../trained_models/"

existing_inference_results = os.listdir("test_results/")

for model_folder in os.listdir(trained_models_fld):
    
    if f"{model_folder}.json" not in existing_inference_results:
        
        print(model_folder)
        
        model_type = get_model_type(model_folder)
        current_test_dataset = test_dataset_dict[model_type]      
        
        test_dataloader = DataLoader(
            current_test_dataset,
            batch_size=64,
            # num_workers=4,
            drop_last=True
        )
        
        model_folder_abs = os.path.join(trained_models_fld, model_folder)        
        test_model = AutoModelForSequenceClassification.from_pretrained(f"{model_folder_abs}/nli_model/")
        test_model = test_model.to(device)

        list_predited_label = []
        list_label = []
        with torch.no_grad():
            for d in tqdm(test_dataloader):
                input_ids = d["input_ids"].to(device) # .reshape(64, 24)
                attention_mask = d["attention_mask"].to(device)

                outputs = test_model(input_ids, attention_mask)
                logits = outputs[0]

                _, prediction = torch.max(logits, dim=1)
                targets = d["labels"].detach().numpy().tolist()
                prediction = prediction.cpu().detach().numpy().tolist()

                list_label.extend(targets)
                list_predited_label.extend(prediction)

            with open(f"test_results/{model_folder}.json", "w") as f:
                json.dump(list_predited_label, f)
    else:
        print("dropped", model_folder)

dropped microsoft_deberta-base_ep10_wus100_lr1e-06
dropped microsoft_deberta-base_ep10_wus1000_lr5e-05
dropped microsoft_deberta-base_ep10_wus2000_lr1e-06
dropped microsoft_deberta-base_ep10_wus2000_lr5e-05
dropped microsoft_deberta-base_ep5_wus1000_lr1e-05
dropped microsoft_deberta-base_ep10_wus2000_lr1e-05
dropped microsoft_deberta-base_ep10_wus1000_lr0.0001
dropped microsoft_deberta-base_ep10_wus1000_lr1e-05
dropped microsoft_deberta-base_ep5_wus100_lr1e-05
dropped microsoft_deberta-base_ep10_wus2000_lr0.0001
dropped microsoft_deberta-base_ep5_wus2000_lr5e-05
dropped microsoft_deberta-base_ep5_wus1000_lr1e-06
dropped microsoft_deberta-base_ep5_wus2000_lr1e-05
dropped microsoft_deberta-large_ep5_wus100_lr0.0001
dropped microsoft_deberta-large_ep5_wus100_lr1e-05
dropped microsoft_deberta-large_ep10_wus100_lr0.0001
dropped microsoft_deberta-base_ep5_wus100_lr1e-06
dropped microsoft_deberta-base_ep10_wus100_lr0.0001
dropped microsoft_deberta-base_ep5_wus100_lr5e-05
dropped microsoft_deb

100%|█████████████████████████████████████████| 650/650 [01:07<00:00,  9.62it/s]

dropped microsoft_deberta-base_ep10_wus100_lr5e-05
dropped microsoft_deberta-base_ep5_wus1000_lr0.0001


In [21]:
# with open(f"GYAFC_test_labels.json", "w") as f:
#     json.dump(list_label, f)

# len(list_label)